<a href="https://colab.research.google.com/github/snpushpi/6.804-Computational-Cognitive-Science/blob/master/Particle_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import math
import scipy.stats
from collections import Counter 

state_set = {'A','B','C','D'}
mu_dict = {'A':.2,'B':.4,'C':.6,'D':.8}
sigma = 0.1
def data_generate(alpha,T):
    '''Goal is generate T observations with state changing probability alpha.
    Step 1: Set Z_0 as a random sample from state list.
    Step 2: Repeat the following for T trials -
          i) Z_i = Z_i-1
          ii)Sample x randomly from [0,1]
          iii) If x<alpha replace Z_i with random sample {A,B,C,D}/Z_i-1
          iv)Sample stimulus y_i form a normal distribution with std sigma and mu_zi
    '''
    observation_list = []
    Z = [None]*(T+1)
    Z[0] = random.choice(tuple(state_set))
    for i in range(1,T+1):
        Z[i]=Z[i-1]
        x = random.uniform(0,1)
        if x<alpha:
            new_set = state_set-{Z[i-1]}
            Z[i]= random.choice(tuple(new_set))
        observation_list.append(random.gauss(mu_dict[Z[i]],sigma))        
    return observation_list,Z[1:]

def particle_filter(observation_list,r,particle_num,T):
    age_dict = {i:{j:None for j in range(particle_num)} for i in range(0,T+1)}
    U = {i:{j:0 for j in range(particle_num)} for i in range(0,T+1)}
    for i in range(particle_num):
        age_dict[0][i]=0
        U[0][i]=random.choice(tuple(state_set))
    for i in range(1,T+1):
        U[i]=U[i-1]
        age_dict[i]=age_dict[i-1]
        p = 0
        while p < particle_num:
            x1 = random.uniform(0,1)
            pow = r*age_dict[i][p]
            if x1< 1-math.exp(-pow):
                new_set = state_set-{U[i][p]}
                U[i][p]=random.choice(tuple(new_set))
                age_dict[i][p]=0
            else:
                age_dict[i][p]+=1
            x2 = random.uniform(0,1)
            new_set = state_set-{U[i][p]}
            Z = random.choice(tuple(new_set))
            if scipy.stats.norm(mu_dict[Z],sigma).pdf(observation_list[i-1])>x2:
                U[i][p]=Z
                age_dict[i][p]=0
                p+=1
    return U

def count_accuracy(alpha,T,particle_num,r):
    observation_list,actual_list = data_generate(alpha,T)
    U = particle_filter(observation_list,r,particle_num,T)
    prediction = []
    for i in range(1,T+1):
        value, count = Counter(U[i].values()).most_common(1)[0]
        prediction.append(value)
    final_count = 0
    for i in range(T):
        if prediction[i]==actual_list[i]:
            final_count+=1
    return actual_list, prediction, final_count/T
print(count_accuracy(0.08,1000,150,0.3))



    



(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'B', 'B', 'D', 'D', 'D', 'D', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'